### radar plot representing the proportion of reduced publication of countries in each discipline after the "internationally funded" (there are multiple removing rules) publications are removed. The input contains:
- raw_path:original number of publications of countries
- filter_path: the number of publication after "international funded" papers are removed
- disgroup_path: map discipline to discplinary level
- discode_path: map code of discipline to discipline name

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import pi
import os

In [3]:
raw_path="../../data/nf_folder/Data/DerivedData/Derived/cntry_author_full.csv"
filter_path='../../data/nf_folder/Data/DerivedData/Derived/dep-FundExchange/pub_no_foreign_funder.pkl'
disgroup_path='../../data/nf_folder/Data/AdditionalData/discipline_classification.csv'
discode_path='../../data/nf_folder/Data/AdditionalData/discipline_code.csv'

In [16]:
raw_df=pd.read_csv(raw_path)
raw_df.head()

,cntry,year,dis,count
0,Afghanistan,2009,3,1
1,Afghanistan,2009,33,1
2,Afghanistan,2009,35,1
3,Afghanistan,2009,44,2
4,Afghanistan,2009,47,1


In [17]:
filter_df=pd.read_csv(filter_path)
filter_df.head()

,cntry,year,dis,count
0,Afghanistan,2009,3,1
1,Afghanistan,2009,33,1
2,Afghanistan,2009,44,2
3,Afghanistan,2009,47,1
4,Afghanistan,2009,52,1


In [6]:
disgroup=pd.read_csv(disgroup_path)
disgroup.head(2)

,abbrev,level_1,level_2,level_3
0,Fine Arts/Arch,Fine Arts & Architecture,Arts,Arts and Humanities
1,Perf. Arts,Performing Arts,Arts,Arts and Humanities


In [7]:
discode=pd.read_csv(discode_path)
discode.head(2)

,Code_Discipline,ESpecialite
0,1,Agricult & Food Science
1,2,Botany


In [10]:
dismeta=disgroup[['level_1','level_2']].merge(discode,left_on='level_1',right_on='ESpecialite')
dismeta=dismeta[['Code_Discipline','level_1','level_2']]
dismeta.columns=['dis','disname','level']

In [25]:
meta=raw_df.merge(filter_df,on=['cntry','year','dis'],suffixes=('_raw', '_filter'),how='left')
meta=meta.fillna(0)
meta=meta.merge(dismeta,on='dis')
meta=meta.groupby(['cntry','level']).agg({'count_raw':'sum','count_filter':'sum'}).reset_index()
meta.head()

,cntry,level,count_raw,count_filter
0,Afghanistan,Biology,53,26.0
1,Afghanistan,Biomedical Research,37,15.0
2,Afghanistan,Chemistry,12,8.0
3,Afghanistan,Clinical Medicine,267,156.0
4,Afghanistan,Earth and Space,17,6.0


In [29]:
dislist=['Biology','Biomedical Research','Clinical Medicine','Physics','Chemistry','Engineering and Technology',
        'Mathematics','Earth and Space']
meta['p']=(meta['count_raw']-meta['count_filter'])/(meta['count_raw'])#calculate decreased ratio
meta=meta.pivot(index='cntry',columns='level',values='p')
meta=meta[dislist]#order columns based on defined list
meta=meta.fillna(0)
meta.head()

level,Biology,Biomedical Research,Clinical Medicine,Physics,Chemistry,Engineering and Technology,Mathematics,Earth and Space
cntry,,,,,,,,
Afghanistan,0.509434,0.594595,0.415730,1.000000,0.333333,0.300000,0.000000,0.647059
Albania,0.385000,0.433824,0.203620,0.240000,0.285714,0.217105,0.081967,0.220544
Algeria,0.245816,0.383365,0.288024,0.205245,0.276243,0.112155,0.184698,0.222435
Andorra,0.666667,0.166667,0.440000,0.000000,0.000000,0.500000,0.000000,0.583333
Angola,0.609195,0.746479,0.555556,1.000000,0.769231,0.347826,0.333333,0.626866


In [30]:
def plot_radar(values,labels):
    fig, ax = plt.subplots(subplot_kw=dict(projection="polar"))
    values += values[:1]
    theta = np.linspace(0, 2 * np.pi, len(values))
    ax.plot(theta, values)
    plt.ylim(0,1)
    plt.yticks([0.2,0.4,0.6,0.8,1])
    plt.thetagrids(range(0, 360, int(360/len(labels))),(labels))
    return fig, ax

In [33]:
labels=['Biology','Biomed.','Clinical Med.','Physics','Chemistry','Eng. & Tech.','Math','Earth & Space']
map_dir="../../data/nf_folder/Figs/dep-FundExchange/profile_change"
for cntry,row in meta.iterrows():
    values=row.values.tolist()
    fig, ax=plot_radar(values,labels)
    plot_path=os.path.join(map_dir, cntry+ '.' + "pdf")
    plt.savefig(plot_path,bbox_inches='tight')
    plt.close()